# Imports

In [181]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
from skimage.color import rgb2hsv
import os

# Constants

In [182]:
DIR_INP = './room_scenes/'
DIR_OUT = './output/'
DIR_TRN = DIR_OUT+'train/'
DIR_TST = DIR_OUT+'test/'
S_RATIO = 0.8

# Functions

In [196]:
# Function to delete all files in the output folder
from cgi import test


def deleteFiles(fpath):
    files = os.listdir(fpath)
    for f in files:
        os.remove(fpath+f)

# Function to perform Segmentation on an Image
def doSeg4Image(inp):
    """
    Perform Segmentation on an Image Array (256,256,3)
    """
    vec = np.uint8(inp.reshape((-1,3)))
    kmn = KMeans(random_state=42, init='random', n_clusters=3)
    lb0 = kmn.fit_predict(vec)
    cl0 = [255,0,0]
    cl1 = [0,255,0]
    cl2 = [0,0,255]
    cpy = np.zeros((lb0.shape[0],3), dtype=np.uint8)
    for i, j in enumerate(lb0):
        if j == 0:
            cpy[i] = cl0
        elif j == 1:
            cpy[i] = cl1
        else:
            cpy[i] = cl2
    out = np.reshape(cpy, (256,256,3))
    return out

#Function to do Segmentation for files in a Folder
def doSegmentation(inputFolder, trainPath, testPath):
    files = os.listdir(inputFolder)
    threshold = round(number=len(files)*S_RATIO)
    for idx, fil in enumerate(files):
        f = trainPath if idx<=threshold else testPath
        im0 = Image.open(fp=inputFolder+fil)
        im1 = im0.resize(size=(256,256))
        inp = np.array(im1)
        out = doSeg4Image(inp)
        fin = np.concatenate((inp,out), axis=1)
        img = Image.fromarray(fin)
        img.save(fp=f+fil)

# Core Logic

In [197]:
# Delete all the files in the output folders
deleteFiles(DIR_TRN)
deleteFiles(DIR_TST)
doSegmentation(DIR_INP, DIR_TRN, DIR_TST)